## Доверительные интервалы

**1.** В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо. 

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [1]:
import numpy as np

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

In [2]:
n1 = 11037
t1 = 104
n2 = 11034
t2 = 189

p1 = t1 / n1
p2 = t2 / n2
print(f'Вероятность инфаркта при приёме аспирина снижается на {np.round(p2 - p1 , 4)}')

Вероятность инфаркта при приёме аспирина снижается на 0.0077


**2.** Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.  

### Доверительный интервал для разности долей (независимые выборки)

Suc | $X_1$ | $X_2$  
--- | --- | ----|
  1  | $a$ | $b$ 
  0  | $c$ | $d$ 
  $\sum$ | $n_1$| $n_2$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

In [3]:
alpha = 0.05
z = scipy.stats.norm.ppf(1 - alpha / 2.)   

left_boundary = (p2 - p1) - z * np.sqrt(p2 * (1 - p2)/ n2 + p1 * (1 - p1)/ n1)
right_boundary = (p2 - p1) + z * np.sqrt(p2 * (1 - p2)/ n2 + p1 * (1 - p1)/ n1)

In [4]:
print(f'Confidence interval: {[round(left_boundary, 4), round(right_boundary, 4)]}')

Confidence interval: [0.0047, 0.0107]


**3.** Продолжим анализировать данные эксперимента Гарвардской медицинской школы. 

Для бернуллиевских случайных величин $X\sim Ber(p)$ часто вычисляют величину $\frac{p}{1-p}$, которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо $p$ нужно подставить $\hat{p}$. Например, шансы инфаркта в контрольной группе, принимавшей плацебо, можно оценить как 

$$ \frac{\frac{189}{11034}}{1-\frac{189}{11034}} = \frac{189}{11034-189}\approx 0.0174 $$

Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки. 

In [5]:
print(f'Шансы инфаркта при регулярном приёме аспирина снижаются на {round((p2/(1-p2)) / (p1/(1-p1)),4)}')

Шансы инфаркта при регулярном приёме аспирина снижаются на 1.8321


**4.** Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:

- составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;
- установите random seed=0;
- сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.

In [6]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [7]:
np.random.seed(0)

group1 = np.append(np.ones(t1), np.zeros(n1-t1))
group2 = np.append(np.ones(t2), np.zeros(n2-t2))

In [8]:
def odds(group):
    p = group.sum() / group.shape[0]
    return p / (1 - p)
odds1 = np.array(list(map(odds, get_bootstrap_samples(group1, 1000))))
odds2 = np.array(list(map(odds, get_bootstrap_samples(group2, 1000))))

In [9]:
print(f'Сonfidence interval for the odds ratio: {np.round(stat_intervals(odds2/odds1, 0.05), 4)}')

Сonfidence interval for the odds ratio: [1.4442 2.3432]


Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины $X\sim N\left(\mu,\sigma^2\right)$ лежит в интервале $\mu\pm c \cdot \sigma$. Чему равно точное значение константы $c$? Округлите ответ до четырёх знаков после десятичной точки.  

In [10]:
print(f'c = z_coef (critical value from the normal distribution) with 99.7% confidence level = {np.round(scipy.stats.norm.isf(0.003/2), 4) }')

c = z_coef (critical value from the normal distribution) with 99.7% confidence level = 2.9677
